In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.stats.proportion import proportion_confint
import numpy as np
from sklearn.linear_model import LogisticRegression

# Suppress pandas warnings
pd.options.mode.chained_assignment = None

In [2]:
# CONFIG
OUT_PATH = "../Week3/Control"

In [3]:
# Load and preprocess Control data
df = pd.read_excel('../Data/control.xlsx')

# Create side win flag for Offense
df['Off_Win'] = (df['Winner'] == df['Offense']).astype(int)

# Parse life differentials
def parse_diff(s):
    try:
        off, defe = s.split('/')
        return int(off) - int(defe)
    except:
        return pd.NA

def get_off_lives(s):
    try:
        return int(s.split('/')[0])
    except:
        return pd.NA

def get_def_lives(s):
    try:
        return int(s.split('/')[1])
    except:
        return pd.NA
    
df['LifeDiff_2Seg'] = df['Off/Def-2T'].apply(parse_diff)
df['LifeDiff_End'] = df['Off/Def_RoundEnd'].apply(parse_diff)
df['OffLivesEnd'] = df['Off/Def_RoundEnd'].apply(get_off_lives)
df['DefLivesEnd'] = df['Off/Def_RoundEnd'].apply(get_def_lives)

In [4]:
# Filter for relevant teams
relevant_teams = ['Q9', 'OUG', 'SPG', 'XROCK', 'GodL', 'Wolves']
df_masters = df[df['Offense'].isin(relevant_teams) | df['Defense'].isin(relevant_teams)]

In [5]:
df_masters

,Date,Map,Offense,Defense,Round,FBTeam,FBTraded,Off/Def-2T,2TickTime,OffTicks,Zone(s) Captures,Off/Def_RoundEnd,Winner,Notes,Off_Win,LifeDiff_2Seg,LifeDiff_End,OffLivesEnd,DefLivesEnd
0,2025-08-06,Takeoff,OUG,Wolves,1,OUG,Yes,21/24,1:28,4,A,8/0,OUG,NaN,1,-3,8,8,0
1,2025-08-06,Takeoff,Wolves,OUG,2,OUG,No,NaN,NaN,0,NaN,2/8,OUG,NaN,0,<NA>,-6,2,8
2,2025-08-06,Takeoff,OUG,Wolves,3,Wolves,No,15/17,1:03,3,A,0/5,Wolves,NaN,0,-2,-5,0,5
3,2025-08-06,Takeoff,Wolves,OUG,4,Wolves,No,NaN,NaN,0,NaN,0/20,OUG,Raph 7 Spree,0,<NA>,-20,0,20
4,2025-08-06,Takeoff,OUG,Wolves,5,OUG,No,8/9,0:40,4,A,2/0,OUG,NaN,1,-1,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2025-08-23,Takeoff,OUG,Q9,1,OUG,No,27/21,1:43,3,A,0/3,Q9,NaN,0,6,-3,0,3
180,2025-08-23,Takeoff,Q9,OUG,2,OUG,Yes,NaN,NaN,0,NaN,2/5,OUG,NaN,0,<NA>,-3,2,5
181,2025-08-23,Takeoff,OUG,Q9,3,OUG,No,3/4,0:07,3,NaN,0/3,Q9,NaN,0,-1,-3,0,3
182,2025-08-23,Takeoff,Q9,OUG,4,OUG,Yes,21/22,1:24,5,B,3/0,Q9,NaN,1,-1,3,3,0


In [6]:
# 1. Off-vs-Def win splits for each map
win_split = df_masters.groupby('Map')['Off_Win'].agg(
    OffenseWins='sum', TotalRounds='count'
).assign(DefenseWins=lambda x: x['TotalRounds'] - x['OffenseWins'],
         OffenseWinRate=lambda x: x['OffenseWins'] / x['TotalRounds'],
         DefenseWinRate=lambda x: x['DefenseWins'] / x['TotalRounds']
).reset_index()

win_split.drop(columns=['OffenseWins', 'DefenseWins', 'TotalRounds'], inplace=True)

display(win_split)

,Map,OffenseWinRate,DefenseWinRate
0,Crossfire,0.414286,0.585714
1,Raid,0.406250,0.593750
2,Takeoff,0.310345,0.689655


In [7]:
# Convert 2TickTime to seconds
def convert_to_seconds(t):
    try:
        minutes, seconds = map(int, t.split(':'))
        return 120 - (minutes * 60 + seconds)
    except:
        return pd.NA

df_masters['TimeTo2Ticks'] = df_masters['2TickTime'].apply(convert_to_seconds)

In [8]:
SEED = 42
N_BOOT = 1000
LIFE_COL = 'LifeDiff_2Seg'
TIME_COL = 'TimeTo2Ticks'

df_clean = df_masters.dropna(subset=[LIFE_COL, TIME_COL, 'Off_Win']).copy()

X_life = df_clean[LIFE_COL].to_numpy().reshape(-1, 1)
X_time = df_clean[TIME_COL].to_numpy().reshape(-1, 1)
y = df_clean['Off_Win'].astype(int).to_numpy()

# Stack features: [LifeDiff, TwoTickTime]
X = np.hstack([X_life, X_time])

# Fit logistic regression on raw rounds (2D)
log_reg = LogisticRegression(solver='lbfgs', max_iter=1000)
log_reg.fit(X, y)

# Build prediction grid along LifeDiff only, holding time at reference(s)
x_min, x_max = X_life.min(), X_life.max()
pad = max(1.0, 0.1 * (x_max - x_min))
life_grid = np.linspace(x_min - pad, x_max + pad, 201)[:, None]

# Time references (for plotting lines): median (main), and optional fast/slow (25th/75th)
t_median = np.median(X_time)
t_p25    = np.percentile(X_time, 25)
t_p75    = np.percentile(X_time, 75)

# Helper to predict over life grid for a fixed time value
def predict_at_time(t_fixed):
    Xg = np.hstack([life_grid, np.full_like(life_grid, fill_value=t_fixed)])
    return log_reg.predict_proba(Xg)[:, 1]

p_med  = predict_at_time(t_median)
p_fast = predict_at_time(t_p25)   # "faster" reach to 2 ticks (lower time)
p_slow = predict_at_time(t_p75)   # "slower" reach to 2 ticks (higher time)

# Bootstrap CIs (for median-time curve)
rng = np.random.default_rng(SEED)
boot_preds = np.full((N_BOOT, life_grid.shape[0]), np.nan, dtype=float)

for i in range(N_BOOT):
    idx = rng.integers(0, len(X), size=len(X))  # sample rows with replacement
    Xb, yb = X[idx], y[idx]
    try:
        m = LogisticRegression(solver='lbfgs', max_iter=1000)
        m.fit(Xb, yb)
        # predict on life grid at median time
        Xg_med = np.hstack([life_grid, np.full_like(life_grid, fill_value=t_median)])
        boot_preds[i, :] = m.predict_proba(Xg_med)[:, 1]
    except Exception:
        # leave this bootstrap row as NaNs on failure (rare with small samples / separation)
        pass

ci_low  = np.nanpercentile(boot_preds,  2.5, axis=0)
ci_high = np.nanpercentile(boot_preds, 97.5, axis=0)

# Save CSV for Datawrapper ---
out = pd.DataFrame({
    'LifeDiff': life_grid.ravel(),
    'WinProb_medTime': p_med,     # main curve (time fixed at median)
    'CI_low': ci_low,             # 95% CI around the median-time curve
    'CI_high': ci_high,
    'WinProb_fastTime': p_fast,   # optional comparison lines (no CI)
    'WinProb_slowTime': p_slow
})

# (Optional) include the numeric time references used (seconds) as columns for clarity
out.attrs = {'t_median': float(t_median), 't_p25': float(t_p25), 't_p75': float(t_p75)}
out.to_csv(f"{OUT_PATH}/life_diff_curve.csv", index=False)

print(f"Saved curve (2-feature model; LifeDiff on x-axis) with CI at median {TIME_COL}: {OUT_PATH}/life_diff_curve.csv")
print(f"Time refs used — median: {t_median:.2f}, p25: {t_p25:.2f}, p75: {t_p75:.2f} (same units as {TIME_COL})")


Saved curve (2-feature model; LifeDiff on x-axis) with CI at median TimeTo2Ticks: ../Week3/Control/life_diff_curve.csv
Time refs used — median: 38.00, p25: 22.50, p75: 60.50 (same units as TimeTo2Ticks)


In [9]:
life_diff_records = []

for team in df_masters['Offense'].unique():
    # Select only the rounds where this team played
    mask = (df_masters['Offense'] == team) | (df_masters['Defense'] == team)
    # Compute differential per round from that team's perspective
    diffs = df_masters.loc[mask].apply(
        lambda r: (r['OffLivesEnd'] - r['DefLivesEnd'])
                  if r['Offense'] == team
                  else (r['DefLivesEnd'] - r['OffLivesEnd']),
        axis=1
    )
    life_diff_records.append({
        'Team': team,
        'CumulativeLifeDiff': diffs.sum(),
        'AvgLifeDiff': diffs.mean(),
        'RoundsPlayed': mask.sum(),
    })

# Create DataFrame of results
life_diff = pd.DataFrame(life_diff_records)

# Filter for relevant teams
life_diff = life_diff[life_diff['Team'].isin(relevant_teams)]

print("4. Round Differential for Each Team:")
display(life_diff.sort_values(by='AvgLifeDiff', ascending=False, ignore_index=True))

4. Round Differential for Each Team:


,Team,CumulativeLifeDiff,AvgLifeDiff,RoundsPlayed
0,Wolves,94,2.410256,39
1,GodL,95,1.900000,50
2,XROCK,44,1.073171,41
3,OUG,48,0.923077,52
4,Q9,-7,-0.122807,57
5,SPG,-159,-3.613636,44


In [10]:
teams = pd.unique(df_masters[['Offense','Defense']].values.ravel())

records = []
for team in teams:
    played = df_masters[(df_masters['Offense']==team)|(df_masters['Defense']==team)]
    wins = (played['Winner']==team).sum()
    losses = len(played) - wins
    records.append({'Team': team, 'RoundDiff': wins - losses})

round_diff = pd.DataFrame(records)

# Filter for relevant teams
round_diff = round_diff[round_diff['Team'].isin(relevant_teams)]
print("Round Differential (Wins − Losses) per Team:")
display(round_diff.sort_values(by='RoundDiff', ascending=False, ignore_index=True))

Round Differential (Wins − Losses) per Team:


,Team,RoundDiff
0,Wolves,11
1,XROCK,7
2,OUG,2
3,GodL,2
4,Q9,1
5,SPG,-14


In [11]:
# Total ticks captured by each team on each map
ticks_off = (
    df_masters.groupby(['Map', 'Offense'], as_index=False)
      .agg(
          TicksCaptured=('OffTicks', 'sum'),
          OffenseRounds=('OffTicks', 'size')
      )
      .assign(AvgTicksPerOffRound=lambda d: d['TicksCaptured'] / d['OffenseRounds'].where(d['OffenseRounds']>0, pd.NA))
      .rename(columns={'Offense':'Team'})
      .sort_values(['Map','TicksCaptured'], ascending=[True, False])
)


# Sort by Map and TicksCaptured
ticks_off = ticks_off.sort_values(by=['Map', 'TicksCaptured'], ascending=[True, False], ignore_index=True)

# Save to CSV
# ticks_off.to_csv('../Week2/Control/ticks_captured.csv', index=False)

In [12]:
ticks_def = (
    df_masters.groupby(['Map', 'Defense'], as_index=False)
        .agg(
            TicksAllowed=('OffTicks', 'sum'),       # opponent's OffTicks while you defend
            DefenseRounds=('OffTicks', 'size')
        )
        .assign(AvgTicksAllowedPerDefRound=lambda d: d['TicksAllowed'] / d['DefenseRounds'].where(d['DefenseRounds']>0, pd.NA))
        .rename(columns={'Defense':'Team'})
)

ticks_profile = (
    pd.merge(ticks_off, ticks_def, on=['Map','Team'], how='outer')
        .fillna({'TicksCaptured':0, 'OffenseRounds':0, 'AvgTicksPerOffRound':0,
                'TicksAllowed':0, 'DefenseRounds':0, 'AvgTicksAllowedPerDefRound':0})
        .sort_values(['Map','TicksCaptured'], ascending=[True, False])
)
# ticks_profile.to_csv('../Week2/Control/ticks_team_profile_by_map.csv', index=False)


In [13]:
ticks_off_overall = (
    df_masters.groupby('Offense', as_index=False)
      .agg(TicksCaptured=('OffTicks','sum'), OffenseRounds=('OffTicks','size'))
      .assign(AvgTicksPerOffRound=lambda d: d['TicksCaptured'] / d['OffenseRounds'].where(d['OffenseRounds']>0, pd.NA))
      .rename(columns={'Offense':'Team'})
      .sort_values('TicksCaptured', ascending=False)
)
ticks_def_overall = (
    df_masters.groupby('Defense', as_index=False)
        .agg(TicksAllowed=('OffTicks','sum'), DefenseRounds=('OffTicks','size'))
        .assign(AvgTicksAllowedPerDefRound=lambda d: d['TicksAllowed'] / d['DefenseRounds'].where(d['DefenseRounds']>0, pd.NA))
        .rename(columns={'Defense':'Team'})
)

ticks_profile_overall = (
    pd.merge(ticks_off_overall, ticks_def_overall, on='Team', how='outer')
        .fillna({'TicksCaptured':0, 'OffenseRounds':0, 'AvgTicksPerOffRound':0,
                'TicksAllowed':0, 'DefenseRounds':0, 'AvgTicksAllowedPerDefRound':0})
        .sort_values('TicksCaptured', ascending=False)
)


In [14]:
ticks_profile_overall = ticks_profile_overall[['Team', 'AvgTicksPerOffRound', 'AvgTicksAllowedPerDefRound']]

# Filter for relevant teams
ticks_profile_overall = ticks_profile_overall[ticks_profile_overall['Team'].isin(relevant_teams)]

display(ticks_profile_overall)

,Team,AvgTicksPerOffRound,AvgTicksAllowedPerDefRound
0,OUG,3.846154,2.730769
1,GodL,3.583333,2.653846
2,Q9,2.964286,2.758621
3,XROCK,3.600000,3.857143
4,SPG,2.409091,3.500000
5,Wolves,2.476190,2.722222


In [15]:
# Merge diffs, round_diff, and win_pct into a single DataFrame
team_stats = pd.merge(life_diff, round_diff, on='Team')
# team_stats = pd.merge(team_stats, win_pct, on='Team')
print("Team Stats (Avg Life Diff and Round Diff):")
display(team_stats)
# display(team_stats.sort_values(by='WinPct', ascending=False, ignore_index=True))
# team_stats.to_csv('../Week2/Control/team_stats.csv', index=False)

Team Stats (Avg Life Diff and Round Diff):


,Team,CumulativeLifeDiff,AvgLifeDiff,RoundsPlayed,RoundDiff
0,OUG,48,0.923077,52,2
1,Wolves,94,2.410256,39,11
2,GodL,95,1.900000,50,2
3,SPG,-159,-3.613636,44,-14
4,Q9,-7,-0.122807,57,1
5,XROCK,44,1.073171,41,7


In [16]:
team_stats = pd.merge(team_stats, ticks_profile_overall, on='Team')
display(team_stats)
team_stats.to_csv(f'{OUT_PATH}/team_stats.csv', index=False)

,Team,CumulativeLifeDiff,AvgLifeDiff,RoundsPlayed,RoundDiff,AvgTicksPerOffRound,AvgTicksAllowedPerDefRound
0,OUG,48,0.923077,52,2,3.846154,2.730769
1,Wolves,94,2.410256,39,11,2.476190,2.722222
2,GodL,95,1.900000,50,2,3.583333,2.653846
3,SPG,-159,-3.613636,44,-14,2.409091,3.500000
4,Q9,-7,-0.122807,57,1,2.964286,2.758621
5,XROCK,44,1.073171,41,7,3.600000,3.857143


In [17]:
# Compare FB vs Non-FB win rates
df_masters['FB_Win'] = (df_masters['FBTeam'] == df_masters['Winner']).astype(int)
fb_comparison = df_masters['FB_Win'].value_counts(normalize=True).rename_axis('FB_Win').reset_index()
fb_comparison['FB_Win'] = fb_comparison['FB_Win'].map({0: 'Non-FB', 1: 'FB'})
fb_comparison['Win Rate'] = fb_comparison['FB_Win'].map({
    'FB': df_masters['FB_Win'].mean(),
    'Non-FB': 1 - df_masters['FB_Win'].mean()
})
fb_comparison = fb_comparison[['FB_Win', 'Win Rate']].sort_values(by='FB_Win')

# Add CI to comparison
fb_comparison['CI_Low'], fb_comparison['CI_Upp'] = proportion_confint(
    fb_comparison['Win Rate'] * len(df_masters), 
    len(df_masters), 
    alpha=0.05, 
    method='wilson'
)
print("FB vs Non-FB Win Rates:")    
display(fb_comparison)
# fb_comparison.to_csv('../Week2/Control/fb_win_rates.csv', index=False)

FB vs Non-FB Win Rates:


,FB_Win,Win Rate,CI_Low,CI_Upp
0,FB,0.6,0.522605,0.672706
1,Non-FB,0.4,0.327294,0.477395


In [18]:
# 5. Zone capture frequencies per map
zone_counts = []
for _, row in df_masters.iterrows():
    z = row['Zone(s) Captures']
    if pd.isna(z):
        continue
    zones = [z] if z in ['A', 'B'] else ['A', 'B']
    for zone in zones:
        zone_counts.append((row['Map'], zone))
zone_df = pd.DataFrame(zone_counts, columns=['Map', 'Zone'])
zone_freq = (zone_df
             .groupby(['Map', 'Zone'])
             .size()
             .reset_index(name='Count')
             .pivot(index='Map', columns='Zone', values='Count')
             .fillna(0)).reset_index()

zone_freq.rename(columns={'A': 'A Captures', 'B': 'B Captures'}, inplace=True)
print("5. Zone Capture Frequencies:")
display(zone_freq)

5. Zone Capture Frequencies:


Zone,Map,A Captures,B Captures
0,Crossfire,59,8
1,Raid,5,20
2,Takeoff,30,4


In [19]:
# Total games played per map
total_games = df_masters['Map'].value_counts().reset_index()
total_games.columns = ['Map', 'TotalRounds']

# Merge zone frequencies with total games
zone_freq = zone_freq.merge(total_games, on='Map')

zone_freq

,Map,A Captures,B Captures,TotalRounds
0,Crossfire,59,8,70
1,Raid,5,20,32
2,Takeoff,30,4,58


In [20]:
# Combine with map win splits
map_summary = win_split.merge(zone_freq, on='Map')
print("Map Summary with Win Splits and Zone Captures:")
display(map_summary)

# Save map summary to CSV
map_summary.to_csv(f"{OUT_PATH}/map_summary.csv", index=False)

Map Summary with Win Splits and Zone Captures:


,Map,OffenseWinRate,DefenseWinRate,A Captures,B Captures,TotalRounds
0,Crossfire,0.414286,0.585714,59,8,70
1,Raid,0.406250,0.593750,5,20,32
2,Takeoff,0.310345,0.689655,30,4,58


In [33]:
# Split by map and see top offense teams by wins
top_offense_by_map = (
    df_masters[df_masters['Off_Win'] == 1]
    .groupby(['Map', 'Offense'], as_index=False)
    .agg(OffenseWins=('Off_Win', 'sum'))
    .sort_values(['Map', 'OffenseWins'], ascending=[True, False], ignore_index=True)
)
top_offense_by_map = top_offense_by_map[top_offense_by_map['Offense'].isin(relevant_teams)]

# print("Top Offense Teams by Map (by Wins):")
# print(top_offense_by_map)


pivoted = top_offense_by_map.pivot_table(
    index='Offense', 
    columns='Map', 
    values='OffenseWins',
    fill_value=0).reset_index()

print("Pivoted Offense Wins by Team and Map:")
display(pivoted)
pivoted.to_csv(f"{OUT_PATH}/top_offense_by_map.csv", index=False)

Pivoted Offense Wins by Team and Map:


Map,Offense,Crossfire,Raid,Takeoff
0,GodL,4,3,3
1,OUG,5,1,3
2,Q9,4,1,6
3,SPG,0,4,0
4,Wolves,2,3,6
5,XROCK,10,0,0


In [28]:
# Highest offense win rates per team
offense_win_rates = (
    df_masters.groupby('Offense', as_index=False)
    .agg(OffenseWins=('Off_Win', 'sum'), OffenseRounds=('Off_Win', 'size'))
    .assign(OffenseWinRate=lambda d: d['OffenseWins'] / d['OffenseRounds'].where(d['OffenseRounds']>0, pd.NA))
    .sort_values('OffenseWinRate', ascending=False, ignore_index=True)
)
offense_win_rates = offense_win_rates[offense_win_rates['Offense'].isin(relevant_teams)]

print("Highest Offense Win Rates per Team:")
display(offense_win_rates)

Highest Offense Win Rates per Team:


,Offense,OffenseWins,OffenseRounds,OffenseWinRate
0,Wolves,11,21,0.523810
1,XROCK,10,20,0.500000
3,GodL,10,24,0.416667
4,Q9,11,28,0.392857
5,OUG,9,26,0.346154
8,SPG,4,22,0.181818
